In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

'\n掛載Google硬碟\n安裝套件\n引用套件\nAPP應用準備\n消息準備\nhandler執行方法設計\n啟動應用\n\n'

In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''

流程解析

'''


'\n\n流程解析\n\n'

In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("NLuJuE98mu9aVvIIfWPrgfq9q5EoVZYP+yWK6W4O+UsYdS/GmMpUYFk7qEM2OrrDXikIV+ZRTA1V/BftphfFFqpzzA8m+ygvmWsZ7iCn0kxank+IemyqfWLa7kYok8WV5d+3cTUXJC24ndjiGG6nDgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("b660a0127b1ab050edfe4f9d9c543b4c")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''
#Line主程式對話入口
# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/2021-AI/LineBot/1020event/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理
  #從 python line-bot-sdk找
  #alt_text:初步檢視文字
  #title:標題
  #text:文字描述
  #actions:按鍵,json檔,line bot designer可以找到
'''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "[::text:]尋找BD",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "開發者，尋求教學",
            "text": "[::text:]求助專家",
            "data": "Data2"
          },
          {
            "type": "uri",
            "label": "打電話",
            "uri": "tel://3345678"
          },
          {
            "type": "uri",
            "label": "開啟相機",
            "uri": "https://line.me/R/nv/camera/",
          },
           #按鍵範本最多只能有四個json,這裡先註解其中一個功能,需要再拿出來用
          # {
          #   "type": "uri",
          #   "label": "開啟地理位置功能",
          #   "uri": "https://line.me/R/nv/location/"
          # }
          #問題紀錄：1.uri打成url:會呈現可以連線,但跑不出東西-或跳"message": "must be specified"
        ],
  )
)


In [ ]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
#用戶輸入more呼叫
template_message_dict = {
  "@more":buttons_template_message,
}

In [ ]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
#用戶收到文字消息時的處理方式
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    #event.message.text用戶傳來的文字內容
    #若有find 特殊符號@就執行下面流程
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    #若無,回應查無字典
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@more")
        )

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
#當用戶點及Postback的按鍵時,會發出PostbackEvent
#要告訴handler 收到PostbackEvent該怎麼辦
@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    # 若data=data1,則回應有專人
    # 若data2,則回應專家
    if (event.postback.data.find('Data1')== 0):
      #寫檔案
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，我們會派專家與您聯繫'
                )
            )
    else:
        pass

In [ ]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7eb0-34-90-75-20.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U5307897c9c8ff435a32a1d50e95ee58e","events":[{"type":"message","message":{"type":"text","id":"14947080107530","text":"@more"},"timestamp":1634751039314,"source":{"type":"user","userId":"Ud28913e45991f8c7531b94f85a9992a1"},"replyToken":"554f828ed86d46f2a8ae10e2a3fc86fa","mode":"active"}]}


127.0.0.1 - - [20/Oct/2021 17:30:40] "POST / HTTP/1.1" 200 -
